# EOO results


The visualization should be replicated starting from Viusalization - Plot; input data has been uploaded to s3://statfungen/ftp_fgc_xqtl/analysis_result/EOO/EOO_reshape_combined_result_withSNP_pip01_95cs_annotation_variant_level.updatedSep9th.RDS

## input

In [189]:
# multi context to EOO input 

library(data.table)
library(tidyverse)

multi_con <- fread('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/data/ROSMAP.exported.toploci.bed.gz')

In [177]:
# Filter rows and pull the lfsr column
multi_con_clean <- multi_con %>%
  filter(lfsr != "", cs_coverage_0.95 > 0, PIP > 0.1)

In [178]:
multi_con_clean %>%   pull(lfsr) %>%
  # split by ";" into matrix
  str_split(";", simplify = TRUE) %>%
  # convert to vector
  unlist() %>%
  # convert to numeric
  as.numeric() %>% .[!is.na(.)] %>%
  # show summary statistics
  summary()

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.000e+00 0.000e+00 1.031e-06 8.171e-04 3.627e-04 1.000e-02 

In [179]:

multi_con_clean_long <- multi_con_clean %>%
  # separate_longer_delim is tidyverse way to split long by delimiter
  separate_longer_delim(event_ID, delim = ";")

In [180]:
multi_con_clean_long <- multi_con_clean_long %>% mutate(event_ID = paste0(event_ID, '_multicontext'))

In [181]:
multi_con_clean_long %>% pull(event_ID) %>% unique %>% paste(.,collapse = '"\n"') %>% cat

AC_DeJager_eQTL_multicontext"
"DLPFC_DeJager_eQTL_multicontext"
"PCC_DeJager_eQTL_multicontext"
"Exc_mega_eQTL_multicontext"
"Ast_Kellis_eQTL_multicontext"
"Ast_mega_eQTL_multicontext"
"Exc_DeJager_eQTL_multicontext"
"Exc_Kellis_eQTL_multicontext"
"Inh_DeJager_eQTL_multicontext"
"Inh_Kellis_eQTL_multicontext"
"Inh_mega_eQTL_multicontext"
"Oli_DeJager_eQTL_multicontext"
"Oli_mega_eQTL_multicontext"
"Ast_DeJager_eQTL_multicontext"
"OPC_Kellis_eQTL_multicontext"
"DLPFC_Bennett_pQTL_multicontext"
"OPC_DeJager_eQTL_multicontext"
"Oli_Kellis_eQTL_multicontext"
"OPC_mega_eQTL_multicontext"
"monocyte_ROSMAP_eQTL_multicontext"
"Mic_Kellis_eQTL_multicontext"
"Mic_mega_eQTL_multicontext"
"Mic_DeJager_eQTL_multicontext

In [58]:
multi_con_clean_long %>%
  group_split(event_ID) %>%
  walk(function(df) {
    id <- unique(df$event_ID)
    file_name <- paste0("~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/data/ROSMAP.", id, ".exported.toploci.tsv")  
    fwrite(df %>% select(`#chr`, end) %>% rename(chr = `#chr`, pos = end) %>% arrange(chr, pos), file_name, sep = '\t')
  })

In the first itration, we found the multi context does not show a better enrichment than single context, we wondered if we should use a version
```
Identify pure cell-specific multi-context, which in principle to keep only variants that:
•	Appear in a single context (event_ID has no ;)
•	And do not appear in any rows where event_ID includes a ; (multi-context bundles)
```

In [182]:
# Identify variant_IDs that are in multi-context bundles
multi_context_variants <- multi_con_clean %>%
 filter(str_detect(event_ID, ";")) %>%
 pull(variant_ID) %>%
 unique()

# Keep only rows where event_ID has no ';' AND variant_ID not in multi-context
pure_cell_specific <- multi_con_clean %>%
 filter(!str_detect(event_ID, ";"), !variant_ID %in% multi_context_variants)

In [185]:
multi_con_clean %>% dim

[1] 41213    14

In [183]:
pure_cell_specific %>% dim

[1] 3126   14

too few to continue EOO analysis with this pure way

## Data preparation

In [190]:
library(tidyverse)
library(data.table)

In [191]:
eoo_res_list <- list.files('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment', 'gz$', full.names = T)

In [194]:
eoo_res_list %>% length

[1] 368

In [195]:
eoo_res <- data.frame()
for(i in seq_along(eoo_res_list)){
    eoo_res  <- rbind(eoo_res, fread(eoo_res_list[i]) %>% mutate(study = basename(eoo_res_list[i]) %>% str_split(., '[.]', simplify = T) %>% .[,1]))

}

In [196]:
fwrite(eoo_res, '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment/ROSMAP_multicontext.all.enrichment_results_summary.tsv.gz')

In [197]:
eoo_res %>% head

Annotation,OR,OR_SE,OR_log2,OR_SE_log2,Enrichment,Enrichment_SE,Enrichment_log2,Enrichment_SE_log2,Enrichment_Z_score,Enrichment_P_value,study
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
LHX2_TSS_H3K27ac,10.478451,0.4956974,3.389354,0.06813028,9.462041,0.4672775,3.242068,0.07111028,20.249298,3.603390e-91,AC_DeJager_eQTL_multicontext
LHX2_enhancer,2.891761,0.2098354,1.531948,0.10443331,2.771898,0.1944439,1.470706,0.10097178,14.255515,4.142337e-46,AC_DeJager_eQTL_multicontext
LHX2_enhancer_atac,6.359157,0.6551940,2.668836,0.14953289,6.200055,0.6254002,2.631929,0.14642127,9.913741,3.628028e-23,AC_DeJager_eQTL_multicontext
LHX2_optimal_peak.H3K27,5.353238,0.2439659,2.420412,0.06586453,4.689886,0.2152761,2.229480,0.06632624,21.785450,3.187966e-105,AC_DeJager_eQTL_multicontext
LHX2_optimal_peak.H3K4me3,10.151530,0.4857885,3.343625,0.06918673,9.274875,0.4537692,3.213245,0.07070101,20.439632,7.429773e-93,AC_DeJager_eQTL_multicontext
LHX2_optimal_peak_IDR_ENCODE.ATAC,7.957453,0.4111946,2.992307,0.07456977,7.451540,0.3673269,2.897455,0.07114755,20.285857,1.714488e-91,AC_DeJager_eQTL_multicontext


## Visualization 

### prepare data

#### load updated multicontext

In [227]:
library(data.table)
library(ggplot2)
library(tidyverse)
library(RColorBrewer)
library(scales)
eoo_res_multicon <- fread( '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment/ROSMAP_multicontext.all.enrichment_results_summary.tsv.gz')

#### load data ####
meta <- fread('~/Work/gpQTL//EOO/data/realease_data_readme.txt')
meta <- meta %>% mutate(context_mod = basename(Path) %>% gsub('.exported.toploci.bed.gz', '',.))

In [229]:
eoo_res_multicon %>% pull(study) %>% unique 

[1] "AC_DeJager_eQTL_multicontext"      "Ast_DeJager_eQTL_multicontext"    
 [3] "Ast_Kellis_eQTL_multicontext"      "Ast_mega_eQTL_multicontext"       
 [5] "DLPFC_Bennett_pQTL_multicontext"   "DLPFC_DeJager_eQTL_multicontext"  
 [7] "Exc_DeJager_eQTL_multicontext"     "Exc_Kellis_eQTL_multicontext"     
 [9] "Exc_mega_eQTL_multicontext"        "Inh_DeJager_eQTL_multicontext"    
[11] "Inh_Kellis_eQTL_multicontext"      "Inh_mega_eQTL_multicontext"       
[13] "Mic_DeJager_eQTL_multicontext"     "Mic_Kellis_eQTL_multicontext"     
[15] "Mic_mega_eQTL_multicontext"        "monocyte_ROSMAP_eQTL_multicontext"
[17] "Oli_DeJager_eQTL_multicontext"     "Oli_Kellis_eQTL_multicontext"     
[19] "Oli_mega_eQTL_multicontext"        "OPC_DeJager_eQTL_multicontext"    
[21] "OPC_Kellis_eQTL_multicontext"      "OPC_mega_eQTL_multicontext"       
[23] "PCC_DeJager_eQTL_multicontext"

#### load all other data

In [230]:
eoo_res_all <- readRDS( '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/EOO_reshape_combined_result_withSNP_pip01_95cs_annotation_variant_level.RDS')
# eoo_res <- readRDS( '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/EOO_original_result_files_withSNP_pip01_95cs_annotation_variant_level.RDS')

In [231]:
eoo_res_all <- eoo_res_all %>% rename(Enrichment_Z_score=Z_enrich, Enrichment_P_value = P_enrich)

In [232]:
eoo_res_all %>% filter(method == 'multicontext') %>% pull(study) %>% unique 

[1] "MSBB"         "ROSMAP"       "ROSMAP_AC"    "ROSMAP_DLPFC" "ROSMAP_Exc"  
[6] "ROSMAP_Inh"   "ROSMAP_Mic"   "ROSMAP_OPC"   "ROSMAP_PCC"

In [233]:
eoo_res_all %>% filter(is.na(method)) %>% pull(study) %>% unique 

[1] "ROSMAP_Ast_multicontext"

#### combine as new data

In [235]:
eoo_res_all_nomulticon <- eoo_res_all %>% filter(method != 'multicontext' | is.na(method))

In [236]:
eoo_res_all <- plyr::rbind.fill(eoo_res_all_nomulticon %>% as.data.frame, eoo_res_multicon %>% as.data.frame %>% mutate(method = 'multicontext') %>% filter(Annotation!='UTR_5_UCSC.to_hg38')) # a duplicate annotation

In [237]:
eoo_res_all %>% head

,Annotation,OR,OR_SE,OR_log2,OR_SE_log2,Enrichment,Enrichment_SE,Enrichment_log2,Enrichment_SE_log2,study,method,annotation,annotation_sets,Enrichment_Z_score,Enrichment_P_value,Z_or,P_or
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,LHX2_TSS_H3K27ac,12.723741,0.7243419,3.669451,0.08187278,11.352247,0.6569629,3.504791,0.08324203,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,15.757733,0.000000e+00,44.81894,0
2,LHX2_enhancer,2.973535,0.2473014,1.572179,0.12043096,2.850839,0.2272655,1.511167,0.11543278,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,8.143953,4.440892e-16,13.05461,0
3,LHX2_enhancer_atac,6.056163,0.7897873,2.598404,0.18915166,5.945633,0.7606750,2.571264,0.18561522,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,6.501637,7.945045e-11,13.73715,0
4,LHX2_optimal_peak.H3K27,6.227424,0.3309881,2.638635,0.07691982,5.346442,0.2765385,2.418487,0.07482234,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,15.717315,0.000000e+00,34.30371,0
5,LHX2_optimal_peak.H3K4me3,13.591981,0.7181150,3.764684,0.07614929,12.169811,0.6565377,3.605135,0.07771441,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,17.013206,0.000000e+00,49.43820,0
6,LHX2_optimal_peak_IDR_ENCODE.ATAC,10.153046,0.5578225,3.343841,0.07882028,9.403917,0.4846550,3.233171,0.07397257,AC_DeJager_eQTL,univariate,AI_MPRA_starrseq_baseline_1000G_EUR_hg38_annotations,AI_MPRA_starrseq,17.340000,0.000000e+00,42.42361,0


In [238]:
saveRDS(eoo_res_all, '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/EOO_reshape_combined_result_withSNP_pip01_95cs_annotation_variant_level.updatedSep9th.RDS')

In [239]:
# # reorganize track df 
# old_track_df <- fread("/Work/Fungen_xQTL/2025/Sep/eoo/EOO/data/categorized_annotation_tracks.csv")
# encode_track_df <- fread("/Work/Fungen_xQTL/2025/Sep/eoo/EOO/data/encode_categories_Ru.csv")

# encode_track_df <- encode_track_df %>% select(V1, V2)
# encode_track_df <- encode_track_df[-1,] %>% mutate(Source = 'ENCODE')
# colnames(encode_track_df) <- c('Track', 'Category', 'Source')
# track_df <- plyr::rbind.fill(old_track_df, encode_track_df)
# fwrite(track_df, "/Work/Fungen_xQTL/2025/Sep/eoo/EOO/data/categorized_annotation_tracks.csv")


### Plot

In [240]:
eoo_res_all <- readRDS('EOO_reshape_combined_result_withSNP_pip01_95cs_annotation_variant_level.updatedSep9th.RDS')

In [261]:
enrichment_data <- eoo_res_all %>% mutate(Annotation = gsub('.to_hg38', '', Annotation)) %>% filter(!str_detect(Annotation,'hg19'))

enrichment_data <- enrichment_data %>%
  mutate(Annotation = ifelse(
    str_detect(Annotation, 'ENCFF038KTR|ENCFF015DGX|ENCFF370ANN'),
    str_replace(Annotation, '\\.bed$', '.bed.gz'),
    Annotation  # keep original if condition not met
  )) %>%
  distinct()

In [262]:
track_df <- fread("categorized_annotation_tracks.csv")

enrichment_data <- merge(enrichment_data, track_df, by.x = 'Annotation', by.y = 'Track', all.x = T)%>% select(-Source) %>% distinct

# enrichment_data <- enrichment_data %>%mutate(Annotation = ifelse(str_detect(Annotation, 'ENCFF'),paste0(Category, "_", Annotation),Annotation))
enrichment_data <- enrichment_data %>%mutate(Annotation = paste0(Category, "|", Annotation))

# Reorder Annotation based on the order of Category
enrichment_data$Annotation <- factor(enrichment_data$Annotation,
                                     levels = unique(enrichment_data$Annotation[order(enrichment_data$Category)]))

In [243]:
enrichment_data %>% head

,Annotation,OR,OR_SE,OR_log2,OR_SE_log2,Enrichment,Enrichment_SE,Enrichment_log2,Enrichment_SE_log2,study,⋯,Enrichment_P_value,Z_or,P_or,Assay,Data_Category,Cell_Type,Category,Annotation_Category,Priority,Notes
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.809502,0.11650544,0.8555927,0.09286834,1.783705,0.11105414,0.8347443,0.08981472,Oli_DeJager_eQTL_multicontext,⋯,4.742789e-58,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory
2,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.737711,0.10442133,0.7971878,0.08647314,1.714913,0.09971817,0.7780193,0.08368707,Oli_Kellis_eQTL_multicontext,⋯,2.767894e-66,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory
3,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.816044,0.11174689,0.8607993,0.08874149,1.790219,0.10689954,0.8400134,0.08612717,OPC_Kellis_eQTL_multicontext,⋯,5.981670e-63,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory
4,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.737365,0.22948146,0.7969008,0.19164379,1.715772,0.21975690,0.7782897,0.18587504,monocyte_ROSMAP_eQTL_multicontext,⋯,5.829208e-15,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory
5,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.769361,0.08809096,0.8232284,0.07188886,1.744402,0.08413339,0.8026526,0.06964766,Exc_DeJager_eQTL_multicontext,⋯,1.718045e-95,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory
6,Transcription_Factor_Binding|1_GATA2_1-Interval-Track,1.604405,0.06880918,0.6820384,0.06174599,1.585853,0.06590343,0.6651996,0.05983695,PCC_DeJager_eQTL_multicontext,⋯,6.061492e-128,NA,NA,,,,Transcription_Factor_Binding,Uncategorized,Exploratory,Not directly mapped; exploratory


#### barplot

In [244]:
# Get original Set2 colors (8)
set2_colors <- brewer.pal(12, "Set3")

# Generate extra colors in a similar style
n_cat <- length(unique(enrichment_data$Category))
if (n_cat > 8) {
  extra_colors <- hue_pal(h = c(30, 300), c = 70, l = 70)(n_cat - 8)
  all_colors <- c(set2_colors, extra_colors)
} else {
  all_colors <- set2_colors[1:n_cat]
}

In [245]:
options(repr.plot.width=20, repr.plot.height=10)


In [246]:
##### seperate figures #####
for(context in unique(enrichment_data$study)){
  print(context)
  dir = meta %>% filter(context_mod == context) %>% pull(Method)
  if(length(dir) == 0) dir = 'Other'
  dat <- enrichment_data %>%
      filter(
        study == context,
        Enrichment_P_value < 0.05
      ) %>%  # Only significant
      group_by(Category) %>%
      arrange(desc(Enrichment_log2)) %>%
      slice_head(n = 10) %>%
      ungroup() %>%
      mutate(
        Annotation = reorder(Annotation, Enrichment_log2),  # Fix order ONCE
        ymin = Enrichment_log2 - Enrichment_SE_log2,
        ymax = Enrichment_log2 + Enrichment_SE_log2
      ) %>%
      distinct(Annotation, .keep_all = TRUE)  # Remove duplicates so bars don't stack
    
    ggplot(dat, aes(x = Annotation, y = Enrichment_log2, fill = Category)) +
      geom_col(stat = "identity") +
      geom_errorbar(aes(ymin = ymin, ymax = ymax), width = 0.3) +
      coord_flip() +
      theme_bw() +
      labs(
        title = paste0("Enrichment across Annotations: ", context),
        x = "Annotation",
        y = "log₂(Enrichment Score)",
        fill = "Annotation Set"
      ) +
      scale_fill_manual(values = all_colors)
  output <- paste0('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/',dir, '/', context, '_enrichment_plot.pdf')
  dir.create(dirname(output), recursive = T)
  ggsave(output, height = 50, width = 20, limitsize = FALSE)
}

[1] "Oli_DeJager_eQTL_multicontext"


Warning message in geom_col(stat = "identity"):
“Ignoring unknown parameters: `stat`”
Warning message in dir.create(dirname(output), recursive = T):
“'/home/rf2872/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/Other' already exists”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”


[1] "Oli_Kellis_eQTL_multicontext"


Warning message in geom_col(stat = "identity"):
“Ignoring unknown parameters: `stat`”
Warning message in dir.create(dirname(output), recursive = T):
“'/home/rf2872/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/Other' already exists”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”


[1] "OPC_Kellis_eQTL_multicontext"


...

Warning message in geom_col(stat = "identity"):
“Ignoring unknown parameters: `stat`”
Warning message in dir.create(dirname(output), recursive = T):
“'/home/rf2872/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/single_context_finemapping' already exists”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”


[1] "Inh_xqtl_only_colocboost"


Warning message in geom_col(stat = "identity"):
“Ignoring unknown parameters: `stat`”
Warning message in dir.create(dirname(output), recursive = T):
“'/home/rf2872/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/Other' already exists”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”


[1] "xqtl_only_colocboost"


Warning message in geom_col(stat = "identity"):
“Ignoring unknown parameters: `stat`”
Warning message in dir.create(dirname(output), recursive = T):
“'/home/rf2872/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/Other' already exists”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”


In [247]:
##### plot top all by category #####
top_num=2
for(top_num in c(1:2)){
  enrichment_data_top_category <- enrichment_data %>%
    filter(Enrichment_P_value < 0.05, !str_detect(Annotation, 'ENCFF')) %>%  # optional: focus on significant enrichments 
    mutate(Annotation = factor(Annotation,
                               levels = unique(Annotation[order(Category)]))) %>%
    group_by(Category, study) %>%
    arrange(desc(Enrichment_log2)) %>%
    slice_head(n = top_num)   # optional: focus on significant enrichments
  
  
  enrichment_data_top_category <- merge(enrichment_data_top_category, meta, by.x = 'study', by.y = 'context_mod', all.x = T) %>% 
    mutate(`Data Type` = ifelse(is.na(`Data Type`), 'Other', `Data Type`),
           Method = ifelse(is.na(Method), 'Other', Method)) %>% 
    mutate(`Data Type` = ifelse(Method == 'single_context_finemapping' & str_detect(Modality, 'Inh|Exc|OPC|Oli|Mic|Ast'), 'snuc_eQTL', `Data Type` ),
           `Data Type` = ifelse(Method == 'multi_gene_finemapping', 'multi_gene', `Data Type` ),
           `Data Type` = ifelse(method == 'multicontext', 'multi_context', `Data Type` ),
           `Data Type` = ifelse(str_detect(study, 'haQTL|mQTL') , 'Epi', `Data Type` ))
  plot_data <- enrichment_data_top_category %>% filter(Method != 'trans_finemapping')
  # plot_data <- enrichment_data_top_category
  
  
  dir <- paste0('condense_top', top_num)
  for (data_type in c(na.omit(unique(plot_data$`Data Type`)), 'all')) {
    # for (data_type in c(na.omit(unique(plot_data$`Data Type`)), 'all')) {
  # for (data_type in c('all')) {
    
    if (data_type == 'all') {
      plot_data_tmp <- plot_data
      # width <- height <- 80
    } else {
      plot_data_tmp <- plot_data %>% filter(`Data Type` == data_type)
      # width <- height <- 40
    }
    
    p <- plot_data_tmp %>%
      ggplot(aes(x = Annotation,
                 y = Enrichment_log2,
                 fill = Category)) +
      geom_col() +
      geom_errorbar(aes(ymin = Enrichment_log2 - Enrichment_SE_log2,
                        ymax = Enrichment_log2 + Enrichment_SE_log2),
                    width = 0.3) +
      facet_wrap(~`Data Type` + study, scales = "free_y") +
      coord_flip() +
      theme_bw() +
      labs(title = "Enrichment across Annotations",
           x = "Annotation",
           y = "log₂(Enrichment Score)",
           fill = "Annotation Set") +
      scale_fill_manual(values = all_colors)
    
    output <- paste0('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/', dir, '/', data_type, '_top',top_num,'_enrichment_plot_combined.pdf')
    dir.create(dirname(output), recursive = TRUE, showWarnings = FALSE)
    
    if (data_type == 'all') {
      width <- 100
      height <- 40 * top_num
    } else {
      # Dynamically calculate size based on the number of facets
      num_facets <- length(unique(plot_data_tmp$`Data Type`)) * length(unique(plot_data_tmp$study))
      if(num_facets > 3) height <- 10 + num_facets * top_num else  height <- 10 + num_facets * top_num/2 # adjust multiplier as needed for better sizing
      if(num_facets > 5) width <- 12 + num_facets * 2.5 else if (num_facets > 3) width <- 12 + num_facets * 4 else width <- 12 + num_facets * 6  # adjust multiplier as needed for better sizing
    }  
    ggsave(output, plot = p, height = height, width = width, limitsize = FALSE)
  }
  
}

Warning message:
“Removed 1 row containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message:
“Removed 6 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message:
“Removed 4 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning

In [248]:
##### plot top all by category for cell type marker #####
top_num=2
for(top_num in 5){
  enrichment_data_top_category <- enrichment_data %>%
    filter(Enrichment_P_value < 0.05, str_detect(Annotation, 'CellType')) %>%  # optional: focus on significant enrichments 
    mutate(Annotation = factor(Annotation,
                               levels = unique(Annotation[order(Category)]))) %>%
    group_by(Category, study) %>%
    arrange(desc(Enrichment_log2)) %>%
    slice_head(n = top_num)   # optional: focus on significant enrichments
  
  
  enrichment_data_top_category <- merge(enrichment_data_top_category, meta, by.x = 'study', by.y = 'context_mod', all.x = T) %>% 
    mutate(`Data Type` = 'LR',
           `Data Type` = ifelse(str_detect(study, 'AC|PCC|DLPFC|MSBB|Knight') , 'eQTL', `Data Type` ),
           `Data Type` = ifelse(str_detect(study, 'Inh|Exc|Oli|OPC|Mic|Ast|monocyte') , 'snuc_eQTL', `Data Type` ),
           `Data Type` = ifelse(str_detect(study, 'pQTL') , 'pQTL', `Data Type` )
           )
  plot_data <- enrichment_data_top_category
  
  
  dir <- paste0('condense_top', top_num)
  for (data_type in c(na.omit(unique(plot_data$`Data Type`)))) {

  # for (data_type in c(na.omit(unique(plot_data$`Data Type`)), 'all')) {

    plot_data_tmp <- plot_data %>% filter(`Data Type` == data_type)

    
    p <- plot_data_tmp %>%
      ggplot(aes(x = Annotation,
                 y = Enrichment_log2,
                 fill = Category)) +
      geom_col() +
      geom_errorbar(aes(ymin = Enrichment_log2 - Enrichment_SE_log2,
                        ymax = Enrichment_log2 + Enrichment_SE_log2),
                    width = 0.3) +
      facet_wrap(~`Data Type` + study, scales = "free_y") +
      coord_flip() +
      theme_bw() +
      labs(title = "Enrichment across Annotations",
           x = "Annotation",
           y = "log₂(Enrichment Score)",
           fill = "Annotation Set") +
      scale_fill_manual(values = all_colors)
    
    output <- paste0('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/', dir, '/', data_type, '_top',top_num,'_withENCFF_enrichment_plot_combined.pdf')
    dir.create(dirname(output), recursive = TRUE, showWarnings = FALSE)
    
      # Dynamically calculate size based on the number of facets
      num_facets <- length(unique(plot_data_tmp$`Data Type`)) * length(unique(plot_data_tmp$study))
      if(num_facets > 3) height <- 10 + num_facets * top_num/4 else  height <- 10 + num_facets * top_num/8 # adjust multiplier as needed for better sizing
      if(num_facets > 5) width <- 12 + num_facets * 2.5 else if (num_facets > 3) width <- 12 + num_facets * 4 else width <- 12 + num_facets * 6  # adjust multiplier as needed for better sizing
    ggsave(output, plot = p, height = height, width = width, limitsize = FALSE)
  }
  
}

Warning message:
“Removed 219 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message:
“Removed 760 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message:
“Removed 49 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“conversion failure on 'log₂(Enrichment Score)' in 'mbcsToSbcs': for ₂ (U+2082)”
Warning message:
“Removed 149 rows containing missing values or values outside the scale range
(`geom_col()`).”
Warning message in grid.Call.graphics(C_text, as.graphi

#### volcano

In [249]:
##### valcano without Inf logP #####
enrichment_data_meta <- merge(enrichment_data, meta, by.x = 'study', by.y = 'context_mod', all.x = T) %>% 
  mutate(`Data Type` = ifelse(is.na(`Data Type`), 'Other', `Data Type`),
         Method = ifelse(is.na(Method), 'Other', Method)) %>% 
  mutate(`Data Type` = ifelse(Method == 'single_context_finemapping' & str_detect(Modality, 'Inh|Exc|OPC|Oli|Mic|Ast'), 'snuc_eQTL', `Data Type` ),
         `Data Type` = ifelse(Method == 'multi_gene_finemapping', 'multi_gene', `Data Type` ),
         `Data Type` = ifelse(method == 'multicontext', 'multi_context', `Data Type` ),
         `Data Type` = ifelse(str_detect(study, 'haQTL|mQTL') , 'Epi', `Data Type` ))
plot_data <- enrichment_data_meta %>% filter(Method != 'trans_finemapping')



plot_data <- enrichment_data_meta 


dir <- 'volcano'
for (data_type in c(na.omit(unique(plot_data$`Data Type`)), 'all')) {
  print(data_type)
  if (data_type == 'all') {
    plot_data_tmp <- plot_data
    # width <- height <- 80
  } else {
    plot_data_tmp <- plot_data %>% filter(`Data Type` == data_type)
    # width <- height <- 40
  }
  p <- plot_data_tmp %>%
    filter(!is.na(Category)) %>%
    filter(!is.na(Enrichment_P_value)) %>%
    mutate(logP = -log10(Enrichment_P_value)) %>%
    # Add a column to identify points to label (top 5 by logP)
    ggplot(aes(x = Enrichment_log2, y = logP, color = logP)) +
    geom_point(aes(size = logP), alpha = 0.6) +
    facet_grid(`Data Type` + study ~Category, scales = "fixed") +
    scale_color_gradientn(colors = c("blue", "grey", "red"),
                          name = "-log10(P)") +
    scale_size_continuous(range = c(1, 6), name = "-log10(P)") +
    theme_bw(base_size = 14) +
    theme(
      panel.grid.minor = element_blank(),
      panel.grid.major = element_line(color = "grey90"),
      panel.background = element_rect(fill = "white", color = NA),
      plot.background = element_rect(fill = "white", color = NA),
      strip.background = element_rect(fill = "grey95", color = "grey70"),
      strip.text = element_text(face = "bold", size = 12),
      axis.text = element_text(color = "black"),
      axis.title = element_text(face = "bold"),
      legend.position = "right",
      legend.background = element_rect(fill = "white", color = NA)
    ) +
    labs(
      title = "Volcano Plot: Enrichment Analysis",
      subtitle = "log2(Enrichment) vs -log10(P-value)",
      x = "log2(Enrichment)",
      y = "-log10(P-value)"
    )
  output <- paste0('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/', dir, '/', data_type,'_enrichment_volcano_plot_combined.pdf')
  dir.create(dirname(output), recursive = TRUE, showWarnings = FALSE)
  
  # if (data_type == 'all') {
  width <- 45
  height <- 2.5 * length(unique(plot_data_tmp$study))
  ggsave(output, plot = p, height = height, width = width, limitsize = FALSE)
}

##### valcano with Inf logP #####


dir <- 'volcano_Inf'
for (data_type in c(na.omit(unique(plot_data$`Data Type`)), 'all')) {
  print(data_type)
  if (data_type == 'all') {
    plot_data_tmp <- plot_data
    # width <- height <- 80
  } else {
    plot_data_tmp <- plot_data %>% filter(`Data Type` == data_type)
    # width <- height <- 40
  }
  # Filter and compute logP
  plot_data_tmp <- plot_data_tmp %>%
    filter(!is.na(Category)) %>%
    filter(!is.na(Enrichment_P_value)) %>%
    mutate(logP = -log10(Enrichment_P_value))
  # Calculate 98th percentile of finite logP
  logP_top <- quantile(
    plot_data_tmp$logP[is.finite(plot_data_tmp$logP)], 
    0.99, 
    na.rm = TRUE
  )
  enrich_top <- quantile(
    plot_data_tmp$Enrichment_log2[is.finite(plot_data_tmp$Enrichment_log2)], 
    0.95, 
    na.rm = TRUE
  )
  # Compute the cap and jitter for Inf values
  finite_logP <- plot_data_tmp$logP[is.finite(plot_data_tmp$logP)]
  cap_base <- max(finite_logP, na.rm = TRUE) + 10
  n_inf <- sum(is.infinite(plot_data_tmp$logP), na.rm = TRUE)
  jitter_values <- runif(n_inf, 0, 5)
  # Counter for jitter values
  plot_data_tmp$jitter_index <- NA
  plot_data_tmp$jitter_index[is.infinite(plot_data_tmp$logP)] <- seq_len(n_inf)
  
  # Apply cap and jitter
  plot_data_tmp <- plot_data_tmp %>%
    mutate(logP = case_when(
      is.infinite(logP) ~ cap_base + jitter_values[jitter_index],
      TRUE ~ logP
    )) %>%
    select(-jitter_index)  # Clean up temporary column
  
  
  p <- plot_data_tmp %>%
    mutate(highlight = logP >= logP_top & Enrichment_log2 > enrich_top) %>%
    ggplot(aes(x = Enrichment_log2, y = logP, color = logP)) +
    geom_point(aes(size = logP), alpha = 0.5) +
    # Add red circle outlines around highlighted points
    geom_point(data = ~filter(., highlight), 
               shape = 21, fill = NA, color = "#CD2626", 
               aes(size = logP), stroke = 1) +
    facet_grid(`Data Type` + study ~ Category, scales = "fixed", 
               labeller = labeller(Category = label_wrap_gen(width = 15))) +
    scale_color_gradientn(colors = c("blue", "grey", "#FF6A6A"),
                          name = "-log10(P)") +
    scale_size_continuous(range = c(1, 6), name = "-log10(P)") +
    coord_cartesian(ylim = c(0, max(plot_data_tmp$logP, na.rm = TRUE) * 1.1)) +
    theme_bw(base_size = 14) +
    theme(
      panel.grid.minor = element_blank(),
      panel.grid.major = element_line(color = "grey90"),
      panel.background = element_rect(fill = "white", color = NA),
      plot.background = element_rect(fill = "white", color = NA),
      strip.background = element_rect(fill = "grey95", color = "grey70"),
      strip.text = element_text(face = "bold", size = 12),
      axis.text = element_text(color = "black"),
      axis.title = element_text(face = "bold"),
      legend.position = "right",
      legend.background = element_rect(fill = "white", color = NA)
    ) +
    labs(
      title = paste0("Volcano Plot: Enrichment Analysis (", data_type, ")"),
      subtitle = "log2(Enrichment) vs -log10(P-value)",
      x = "log2(Enrichment)",
      y = "-log10(P-value)",
      caption = "Red circles: Top 1% -log10(P) & Top 5% log2(Enrichment) \nP=0 capped at max finite -log10(P) + 10 with jitter"
    )
  
  output <- paste0('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/enrichment_figures_category/', dir, '/', data_type,'_enrichment_volcano_plot_combined.pdf')
  dir.create(dirname(output), recursive = TRUE, showWarnings = FALSE)
  
  width <- 45
  height <- 2.5 * length(unique(plot_data_tmp$study))
  ggsave(output, plot = p, height = height, width = width, limitsize = FALSE)
}

saveRDS(enrichment_data_meta, '~/Work/Fungen_xQTL/2025/Sep/eoo/EOO/EOO_all_enrichment_withmeta.rds')

[1] "eQTL"
[1] "multi_context"
[1] "Other"
[1] "snuc_eQTL"
[1] "pQTL"
[1] "multi_gene"
[1] "sQTL"
[1] "Epi"
[1] "metabolome"
[1] "all"
[1] "eQTL"
[1] "multi_context"
[1] "Other"
[1] "snuc_eQTL"
[1] "pQTL"
[1] "multi_gene"
[1] "sQTL"
[1] "Epi"
[1] "metabolome"
[1] "all"


### test between methods ####

In [250]:
# test data 

# To identify annotations with significantly different enrichment values between the single-context (eQTL) and multi-gene (adj_eQTL) datasets, 
# a statistical test was performed on annotations with significant enrichment (Enrichment_P_value < 0.05) in at least one dataset. 
# The datasets were merged by matching annotations, retaining only those significant in either dataset, and rows with missing or infinite values were excluded. 
# Due to the non-normal distribution of enrichment differences (confirmed by a Shapiro-Wilk p-value of 5.623531e-60 in one context), 
# a Z-score test was applied to each paired annotation, calculating the difference (Enrichment_adj - Enrichment_pqtl) divided by the combined standard error.
# P-values were derived from Z-scores and adjusted for multiple testing using the False Discovery Rate (FDR) method. 
# Annotations with FDR-adjusted p-values < 0.05 were considered significantly different, 
# indicating systematic differences in enrichment between the single-context and multi-gene datasets for those annotations.

#### multi gene to single context #####

In [251]:
bulk_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'eQTL')
snuc_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'snuc_eQTL')
eQTL_raw <- rbind(bulk_eQTL, snuc_eQTL)
multi_gene_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'multi_gene')

# build a modalities dict to combine single context and multi gene results 
modality_dict <- c(
  # MSBB
  "BM_10_MSBB_eQTL" = "MSBB_BM_10_eQTL",
  "BM_22_MSBB_eQTL" = "MSBB_BM_22_eQTL",
  "BM_36_MSBB_eQTL" = "MSBB_BM_36_eQTL",
  "BM_44_MSBB_eQTL" = "MSBB_BM_44_eQTL",
  
  # DeJager and mega (ROSMAP)
  "AC_DeJager_eQTL" = "ROSMAP_AC_DeJager_eQTL",
  "Ast_DeJager_eQTL" = "ROSMAP_Ast_DeJager_eQTL",
  "Ast_mega_eQTL" = "ROSMAP_Ast_mega_eQTL",
  "DLPFC_DeJager_eQTL" = "ROSMAP_DLPFC_DeJager_eQTL",
  "Exc_DeJager_eQTL" = "ROSMAP_Exc_DeJager_eQTL",
  "Exc_mega_eQTL" = "ROSMAP_Exc_mega_eQTL",
  "Inh_DeJager_eQTL" = "ROSMAP_Inh_DeJager_eQTL",
  "Inh_mega_eQTL" = "ROSMAP_Inh_mega_eQTL",
  "Mic_DeJager_eQTL" = "ROSMAP_Mic_DeJager_eQTL",
  "Mic_mega_eQTL" = "ROSMAP_Mic_mega_eQTL",
  "monocyte_ROSMAP_eQTL" = "ROSMAP_monocyte_ROSMAP_eQTL",
  "Oli_DeJager_eQTL" = "ROSMAP_Oli_DeJager_eQTL",
  "Oli_mega_eQTL" = "ROSMAP_Oli_mega_eQTL",
  "OPC_DeJager_eQTL" = "ROSMAP_OPC_DeJager_eQTL",
  "OPC_mega_eQTL" = "ROSMAP_OPC_mega_eQTL",
  "PCC_DeJager_eQTL" = "ROSMAP_PCC_DeJager_eQTL"
)
eQTL <- eQTL_raw %>% filter(study %in% names(modality_dict)) %>% 
  mutate(study = recode(study, !!!modality_dict))

In [252]:
 
contexts <- modality_dict %>% as.character()

combined_results <- map_dfr(contexts, function(context) {
  # Step 1: Filter significant annotations (Enrichment_P_value < 0.05 in at least one dataset)
  significant_single <- eQTL %>% 
    filter(study == context, Enrichment_P_value < 0.05) %>% 
    select(Annotation)
  significant_multi <- multi_gene_eQTL %>% 
    filter(study == context, Enrichment_P_value < 0.05) %>% 
    select(Annotation)
  significant_annotations <- unique(c(significant_single$Annotation, significant_multi$Annotation))
  
  # Step 2: Merge datasets for significant annotations
  merged_data <- inner_join(
    eQTL %>% 
      filter(study == context, Annotation %in% significant_annotations) %>% 
      select(Annotation, Enrichment_single_context = Enrichment, Enrichment_SE_single_context = Enrichment_SE, Enrichment_P_value_single_context = Enrichment_P_value),
    multi_gene_eQTL %>% 
      filter(study == context, Annotation %in% significant_annotations) %>% 
      select(Annotation, Enrichment_multi_gene = Enrichment, Enrichment_SE_multi_gene = Enrichment_SE, Enrichment_P_value_multi_gene = Enrichment_P_value),
    by = "Annotation"
  )  %>% 
    filter(if_all(starts_with("Enrichment"), is.finite))%>%
    filter(!str_detect(Annotation, '_OTHER')) %>% 
    mutate(
      # Annotation = gsub('.bed.gz', '', Annotation), 
           Annotation = gsub('.bed', '', Annotation) %>% gsub('formatted_output_', '', .) ,
           # , 
           # Annotation = gsub('Transcription_Factor_', 'TF_', Annotation), 
           # Annotation = gsub('Other_Functional_Annotations', 'Others', Annotation), 
           # Annotation = gsub('Chromatin_Interaction_Loops', 'CILs', Annotation), 
           # Annotation = gsub('.HuFNSC02.narrowPeak.lifted', '', Annotation), 
           # Annotation = gsub('_lifted', '', Annotation), 
           # Annotation = gsub('UCSF−UBC.', '', Annotation),  
           # Annotation = gsub('_hg38lift.', '', Annotation), 
           # Annotation = gsub('_SuperEnhancer', '', Annotation), 
           Annotation = gsub('_OTHER_significant', '', Annotation)
           )
  
  # Step 3: Check if there are valid paired observations
  if (nrow(merged_data) == 0) {
    message("No valid paired observations for context: ", context)
    return(NULL)
  }
  
  # Step 4: Perform Z-score test and calculate differences
  results <- merged_data %>%
    mutate(
      Context = context,
      Enrichment_Diff = Enrichment_multi_gene - Enrichment_single_context,
      Enrichment_SE_combined = sqrt(Enrichment_SE_single_context^2 + Enrichment_SE_multi_gene^2),
      z_score = Enrichment_Diff / Enrichment_SE_combined,
      p_value = 2 * pnorm(-abs(z_score)),
      p_adjusted = p.adjust(p_value, method = "fdr"),
      Significance_Status = case_when(
        Enrichment_P_value_single_context < 0.05 & Enrichment_P_value_multi_gene < 0.05 ~ "Both",
        Enrichment_P_value_single_context < 0.05 ~ "Single Context Only",
        Enrichment_P_value_multi_gene < 0.05 ~ "Multi-Gene Only"
      )
    ) %>%
    select(Context, Annotation, Enrichment_single_context, Enrichment_multi_gene,
           Enrichment_Diff, Enrichment_SE_single_context, Enrichment_SE_multi_gene,
           Enrichment_P_value_single_context, Enrichment_P_value_multi_gene, Significance_Status,
           p_value, p_adjusted)
  
  # Step 5: Filter significant results (FDR < 0.05)
  significant <- results %>%
    filter(!is.na(p_adjusted), p_adjusted < 0.05) %>%
    mutate(Category = str_split(Annotation, "\\|", simplify = TRUE)[, 1])%>% distinct(Annotation, .keep_all = T)
  
  if (nrow(significant) > 0) {
    plot_data <- significant %>%
      arrange(desc(abs(Enrichment_Diff))) %>%
      slice_head(n = 30) %>%
      mutate(Annotation = factor(Annotation, levels = Annotation))
    
    p <- ggplot(plot_data, aes(x = Enrichment_Diff, y = Annotation, fill = Category)) +
      geom_bar(stat = "identity") +
      geom_vline(xintercept = 0, linetype = "dashed") +
      labs(
        title = paste0("Top Enrichment Differences for ", context),
        x = "Enrichment Difference (multi_gene - single_context)", y = ""
      ) +
      theme_minimal() +
      theme(plot.title = element_text(hjust = 0.5)) +
      scale_fill_manual(values = all_colors)
    
    out_path <- file.path('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO//enrichment_figures_category', 'methods_compare/multi_gene/', paste0(context, '_multigene_single_enrichment_plot_combined.pdf'))
    dir.create(dirname(out_path), recursive = TRUE, showWarnings = FALSE)
    ggsave(out_path, plot = p, height = 10, width = 14, limitsize = FALSE)
  }
  
  results
})

Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 209 of `x` matches multiple rows in `y`.
ℹ Row 407 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”
Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 112 of `x` matches multiple rows in `y`.
ℹ Row 87 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”
Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 17 of `x` matches multiple rows in `y`.
ℹ Row 698 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expec

In [253]:
significant_results <- combined_results %>%
  filter(!is.na(p_adjusted), p_adjusted < 0.05) %>%
  mutate(Category = str_split(Annotation, "\\|", simplify = TRUE)[, 1])

cat("Number of significantly different annotations (FDR < 0.05):", nrow(significant_results), "\n")

top_num <- 2
plot_data <- significant_results %>%
  group_by(Context, Category) %>%
  arrange(desc(abs(Enrichment_Diff))) %>%
  slice_head(n = top_num) %>%
  ungroup() %>%
  mutate(Annotation = factor(Annotation, levels = unique(Annotation)))

p <- ggplot(plot_data, aes(x = Enrichment_Diff, y = Annotation, fill = Enrichment_Diff > 0)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = c("skyblue", "salmon"), guide = "none") +
  geom_vline(xintercept = 0, linetype = "dashed") +
  labs(
    title = "Top Enrichment Differences for Significant Annotations (FDR < 0.05)",
    x = "Enrichment Difference (multi_gene - single_context)", y = ""
  ) +
  facet_wrap(~Context, scales = "free_y") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5), strip.text = element_text(face = "bold"))

out_path <- file.path('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO//enrichment_figures_category', 'methods_compare/multi_gene/', 'all_multigene_single_enrichment_plot_combined.pdf')
dir.create(dirname(out_path), recursive = TRUE, showWarnings = FALSE)
ggsave(out_path, plot = p, height = 1.5 * length(unique(plot_data$Context)), width = 60, limitsize = FALSE)

Number of significantly different annotations (FDR < 0.05): 3414 


#### multi context to single context #####

In [254]:
enrichment_data_meta %>% filter(`Data Type` == 'multi_context') %>% pull(study) %>% unique 

[1] "AC_DeJager_eQTL_multicontext"      "Ast_DeJager_eQTL_multicontext"    
 [3] "Ast_Kellis_eQTL_multicontext"      "Ast_mega_eQTL_multicontext"       
 [5] "DLPFC_Bennett_pQTL_multicontext"   "DLPFC_DeJager_eQTL_multicontext"  
 [7] "Exc_DeJager_eQTL_multicontext"     "Exc_Kellis_eQTL_multicontext"     
 [9] "Exc_mega_eQTL_multicontext"        "Inh_DeJager_eQTL_multicontext"    
[11] "Inh_Kellis_eQTL_multicontext"      "Inh_mega_eQTL_multicontext"       
[13] "Mic_DeJager_eQTL_multicontext"     "Mic_Kellis_eQTL_multicontext"     
[15] "Mic_mega_eQTL_multicontext"        "monocyte_ROSMAP_eQTL_multicontext"
[17] "Oli_DeJager_eQTL_multicontext"     "Oli_Kellis_eQTL_multicontext"     
[19] "Oli_mega_eQTL_multicontext"        "OPC_DeJager_eQTL_multicontext"    
[21] "OPC_Kellis_eQTL_multicontext"      "OPC_mega_eQTL_multicontext"       
[23] "PCC_DeJager_eQTL_multicontext"

In [255]:
# Filter eQTL data
bulk_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'eQTL')
snuc_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'snuc_eQTL')
eQTL_raw <- rbind(bulk_eQTL, snuc_eQTL)
multi_context_eQTL <- enrichment_data_meta %>% filter(`Data Type` == 'multi_context')

# Build a modalities dict to combine single context and multi gene results
modality_dict_multicontext <- c(
  "AC_DeJager_eQTL" = "AC_DeJager_eQTL_multicontext",
  "DLPFC_DeJager_eQTL" = "DLPFC_DeJager_eQTL_multicontext",
  "PCC_DeJager_eQTL" = "PCC_DeJager_eQTL_multicontext",
  "Ast_DeJager_eQTL" = "Ast_DeJager_eQTL_multicontext",
  "Inh_DeJager_eQTL" = "Inh_DeJager_eQTL_multicontext",
  "Exc_DeJager_eQTL" = "Exc_DeJager_eQTL_multicontext",
  "Mic_DeJager_eQTL" = "Mic_DeJager_eQTL_multicontext",
  "OPC_DeJager_eQTL" = "OPC_DeJager_eQTL_multicontext",
  "Oli_DeJager_eQTL" = "Oli_DeJager_eQTL_multicontext",
  "PCC_Kellis_eQTL" = "PCC_Kellis_eQTL_multicontext",
  "Ast_Kellis_eQTL" = "Ast_Kellis_eQTL_multicontext",
  "Inh_Kellis_eQTL" = "Inh_Kellis_eQTL_multicontext",
  "Exc_Kellis_eQTL" = "Exc_Kellis_eQTL_multicontext",
  "Mic_Kellis_eQTL" = "Mic_Kellis_eQTL_multicontext",
  "OPC_Kellis_eQTL" = "OPC_Kellis_eQTL_multicontext",
  "Oli_Kellis_eQTL" = "Oli_Kellis_eQTL_multicontext",
  "PCC_mega_eQTL" = "PCC_mega_eQTL_multicontext",
  "Ast_mega_eQTL" = "Ast_mega_eQTL_multicontext",
  "Inh_mega_eQTL" = "Inh_mega_eQTL_multicontext",
  "Exc_mega_eQTL" = "Exc_mega_eQTL_multicontext",
  "Mic_mega_eQTL" = "Mic_mega_eQTL_multicontext",
  "OPC_mega_eQTL" = "OPC_mega_eQTL_multicontext",
  "Oli_mega_eQTL" = "Oli_mega_eQTL_multicontext",
  "DLPFC_Bennett_pQTL" = "DLPFC_Bennett_pQTL_multicontext",
  "monocyte_ROSMAP_eQTL" = "monocyte_ROSMAP_eQTL_multicontext"

)

# Harmonize study names
eQTL <- eQTL_raw %>%
  filter(study %in% names(modality_dict_multicontext)) %>%
  mutate(study = recode(study, !!!modality_dict_multicontext))

multi_context_eQTL <- multi_context_eQTL %>%
  filter(study %in% as.character(modality_dict_multicontext))

# Compare enrichment across modalities
contexts <- as.character(modality_dict_multicontext)
combined_results <- map_dfr(contexts, function(context) { 
  significant_single <- eQTL %>% 
    filter(study == context, Enrichment_P_value < 0.05) %>% 
    select(Annotation) 
  significant_multi <- multi_context_eQTL %>% 
    filter(study == context, Enrichment_P_value < 0.05) %>% 
    select(Annotation) 
  significant_annotations <- unique(c(significant_single$Annotation, significant_multi$Annotation)) 
  
  # Step 2: Merge datasets for significant annotations 
  merged_data <- inner_join( 
    eQTL %>% 
      filter(study == context, Annotation %in% significant_annotations) %>% 
      select(Annotation, Enrichment_single_context = Enrichment, Enrichment_SE_single_context = Enrichment_SE, Enrichment_P_value_single_context = Enrichment_P_value), 
    multi_context_eQTL %>% 
      filter(study == context, Annotation %in% significant_annotations) %>% 
      select(Annotation, Enrichment_multi_gene = Enrichment, Enrichment_SE_multi_gene = Enrichment_SE, Enrichment_P_value_multi_gene = Enrichment_P_value), 
    by = "Annotation" 
  ) %>% 
    filter(if_all(starts_with("Enrichment"), is.finite)) 
  
  # Step 3: Check if there are valid paired observations 
  if (nrow(merged_data) == 0) { 
    message("No valid paired observations for context: ", context) 
    return(NULL) 
  } 
  
  # Step 4: Perform Z-score test and calculate differences 
  results <- merged_data %>% 
    mutate( 
      Context = context, 
      Enrichment_Diff = Enrichment_multi_gene - Enrichment_single_context, 
      Enrichment_SE_combined = sqrt(Enrichment_SE_single_context^2 + Enrichment_SE_multi_gene^2), 
      z_score = Enrichment_Diff / Enrichment_SE_combined, 
      p_value = 2 * pnorm(-abs(z_score)), 
      p_adjusted = p.adjust(p_value, method = "fdr"), 
      Significance_Status = case_when( 
        Enrichment_P_value_single_context < 0.05 & Enrichment_P_value_multi_gene < 0.05 ~ "Both", 
        Enrichment_P_value_single_context < 0.05 ~ "Single Context Only", 
        Enrichment_P_value_multi_gene < 0.05 ~ "Multi-Gene Only" 
      ) 
    ) %>% 
    select(Context, Annotation, Enrichment_single_context, Enrichment_multi_gene, 
           Enrichment_Diff, Enrichment_SE_single_context, Enrichment_SE_multi_gene, 
           Enrichment_P_value_single_context, Enrichment_P_value_multi_gene, Significance_Status, 
           p_value, p_adjusted) 
  
  # Step 5: Filter significant results (FDR < 0.05) 
  significant <- results %>% 
    filter(!is.na(p_adjusted), p_adjusted < 0.05) %>% 
    mutate(Category = ifelse(grepl("\\|", Annotation), 
                             str_split(Annotation, "\\|", simplify = TRUE)[, 1], 
                             Annotation))  %>% distinct(Annotation, .keep_all = T)

  if (nrow(significant) > 0) { 
    plot_data <- significant %>% 
      arrange(desc(abs(Enrichment_Diff))) %>% 
      slice_head(n = 30) %>% 
      mutate(Annotation = factor(Annotation, levels = Annotation)) 
    
    p <- ggplot(plot_data, aes(x = Enrichment_Diff, y = Annotation, fill = Category)) + 
      geom_bar(stat = "identity") + 
      geom_vline(xintercept = 0, linetype = "dashed") + 
      labs( 
        title = paste0("Top Enrichment Differences for ", context), 
        x = "Enrichment Difference (multi_context - single_context)", y = "" 
      ) + 
      theme_minimal() + 
      theme(plot.title = element_text(hjust = 0.5)) + 
      scale_fill_manual(values = all_colors) 
    
    out_path <- file.path('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO//enrichment_figures_category', 
                          'methods_compare/multi_context/', paste0(context, '_multigene_single_enrichment_plot_combined.pdf')) 
    dir.create(dirname(out_path), recursive = TRUE, showWarnings = FALSE) 
    ggsave(out_path, plot = p, height = 10, width = 14, limitsize = FALSE) 
  } 
  
  results 
})

# Filter for significant results and plot summary
significant_results <- combined_results %>%
  filter(!is.na(p_adjusted), p_adjusted < 0.05) %>%
  mutate(Category = str_extract(Annotation, "^[^|]+"))

cat("Number of significantly different annotations (FDR < 0.05):", nrow(significant_results), "\n")

top_num <- 2
plot_data <- significant_results %>%
  group_by(Context, Category) %>%
  arrange(desc(abs(Enrichment_Diff))) %>%
  slice_head(n = top_num) %>%
  ungroup() %>%
  mutate(Annotation = factor(Annotation, levels = unique(Annotation)))

p <- ggplot(plot_data, aes(x = Enrichment_Diff, y = Annotation, fill = Enrichment_Diff > 0)) +
  geom_bar(stat = "identity") +
  scale_fill_manual(values = c("skyblue", "salmon"), guide = "none") +
  geom_vline(xintercept = 0, linetype = "dashed") +
  labs(
    title = "Top Enrichment Differences for Significant Annotations (FDR < 0.05)",
    x = "Enrichment Difference (multi_context - single_context)", y = ""
  ) +
  facet_wrap(~Context, scales = "free_y") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5), strip.text = element_text(face = "bold"))

out_path <- file.path('~/Work/Fungen_xQTL/2025/Sep/eoo/EOO//enrichment_figures_category',
                      'methods_compare/multi_context/', 'all_multigene_single_enrichment_plot_combined.pdf')
dir.create(dirname(out_path), recursive = TRUE, showWarnings = FALSE)
ggsave(out_path, plot = p, height = 2.3 * length(unique(plot_data$Context)), width = 18, limitsize = FALSE)

Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 51 of `x` matches multiple rows in `y`.
ℹ Row 1349 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”
Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 49 of `x` matches multiple rows in `y`.
ℹ Row 218 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”
Warning message in inner_join(eQTL %>% filter(study == context, Annotation %in% :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 25 of `x` matches multiple rows in `y`.
ℹ Row 465 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expec

Number of significantly different annotations (FDR < 0.05): 2487 


In [258]:
ggsave(out_path, plot = p, height = 1.3 * length(unique(plot_data$Context)), width = 50, limitsize = FALSE)

## Summary of Updated Multi-Context EOO Results
	1.	The updated results include more contexts than the previous version (Apr 2025).
	2.	Overall patterns remain similar, especially when comparing across AC.
    3.  Subset of annotations shows significant differences in enrichment between the multi-context and single-context models (FDR < 0.05).
	4.	The magnitude and direction of enrichment differences vary across annotations.
	5.	Some annotations (e.g., PU1 promoter/enhancer in microglia) show higher enrichment in the multi-context model.
	6.	Others (e.g., LHX2 promoter in astrocytes) show higher enrichment in the single-context model.
	7.	The pattern of differences is not consistent across cell types or annotation categories.
 
The visualization should be replicated starting from Viusalization - Plot; input data has been uploaded to s3://statfungen/ftp_fgc_xqtl/analysis_result/EOO/EOO_reshape_combined_result_withSNP_pip01_95cs_annotation_variant_level.updatedSep9th.RDS